In [1]:
from ogb.nodeproppred import NodePropPredDataset
from ogb.graphproppred import GraphPropPredDataset
from sklearn.model_selection import ParameterGrid
%reload_ext autoreload
%autoreload 2
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import convo as c 
import convL 
import modularized as m 
import batching as b 
import poolingL
import gin
import math 
import funcy as fy
import gridSearch as g 
import json 
import os



/home/olga/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-18 20:04:10.471688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 20:04:12.546450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-18 20:04:12.546662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [37]:
import sys
sys.path.append("/content/drive/Othercomputers/Mein PC/Completed")

In [2]:
dataset1 = GraphPropPredDataset(name = 'ogbg-molhiv')

In [3]:
dataset_arxiv = NodePropPredDataset(name = 'ogbn-arxiv')

In [8]:
dataset_arxiv[0]

({'edge_index': array([[104447,  15858, 107156, ...,  45118,  45118,  45118],
         [ 13091,  47283,  69161, ..., 162473, 162537,  72717]]),
  'edge_feat': None,
  'node_feat': array([[-0.057943, -0.05253 , -0.072603, ...,  0.173364, -0.172796,
          -0.140059],
         [-0.1245  , -0.070665, -0.325202, ...,  0.068524, -0.372111,
          -0.301036],
         [-0.080242, -0.023328, -0.183787, ...,  0.109919,  0.117589,
          -0.139883],
         ...,
         [-0.22053 , -0.036568, -0.402199, ...,  0.11336 , -0.161393,
          -0.145171],
         [-0.138236,  0.040885, -0.251811, ..., -0.08929 , -0.041253,
          -0.376132],
         [-0.029875,  0.268417, -0.161124, ...,  0.120807,  0.077647,
          -0.091018]], dtype=float32),
  'node_year': array([[2013],
         [2015],
         [2014],
         ...,
         [2020],
         [2020],
         [2020]]),
  'num_nodes': 169343},
 array([[ 4],
        [ 5],
        [28],
        ...,
        [10],
        [ 4],
 

In [10]:
param_grid = {'num_layers': [1, 2, 3, 4, 5, 6],
              'learning_rate': [0.0001, 0.001, 0.01],
              'optimizer': ['Adam'],
              'regularization': ['DropOut', 'NodeSampling', 'DropEdge', 'GDC'],
              'probability': [0.3, 0.5, 0.7],
              'activation': ['relu', 'sigmoid', 'tanh'],
              'units': [32, 64],
              'convo_type': ['gcn', 'gin']}

override_conv_type = 'gin' # None

if override_conv_type is not None:
    param_grid['convo_type'] = [override_conv_type]

small_sample = list(ParameterGrid(param_grid))[0:1]
sample = list(ParameterGrid(param_grid))[0:5]
big_sample = list(ParameterGrid(param_grid))[0:20]
small_sample

[{'activation': 'relu',
  'convo_type': 'gin',
  'learning_rate': 0.0001,
  'num_layers': 1,
  'optimizer': 'Adam',
  'probability': 0.3,
  'regularization': 'DropOut',
  'units': 32}]

In [12]:
ds = GraphPropPredDataset('ogbg-molhiv')

In [13]:
ds[0]

({'edge_index': array([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
           6,  9,  4, 10, 10, 11, 11, 12, 12, 13, 11, 14, 14, 15, 15, 16,
          16, 17, 15, 18,  9,  2, 18,  4],
         [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  5,  7,  6,  8,  7,
           9,  6, 10,  4, 11, 10, 12, 11, 13, 12, 14, 11, 15, 14, 16, 15,
          17, 16, 18, 15,  2,  9,  4, 18]]),
  'edge_feat': array([[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [1, 0, 0],
         [1, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         

In [14]:
g.create_grid_models(sample,'ogbg-molhiv')

[0] Evaluating hparams: {'activation': 'relu', 'convo_type': 'gin', 'learning_rate': 0.0001, 'num_layers': 1, 'optimizer': 'Adam', 'probability': 0.3, 'regularization': 'DropOut', 'units': 32}
Epoch 1/3
138/138 [==============================] - 3s 9ms/step - loss: 0.6824 - auc_1: 0.5595 - binary_accuracy: 0.8184 - precision_1: 0.0375 - recall_1: 0.1923 - val_loss: 0.3528 - val_auc_1: 0.4997 - val_binary_accuracy: 0.9684 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/3
138/138 [==============================] - 1s 6ms/step - loss: 0.2411 - auc_1: 0.5000 - binary_accuracy: 0.9684 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00 - val_loss: 0.1735 - val_auc_1: 0.5000 - val_binary_accuracy: 0.9684 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/3
138/138 [==============================] - 1s 4ms/step - loss: 0.1459 - auc_1: 0.5000 - binary_accuracy: 0.9684 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
[1] Evaluating hparams: {'activation': 'relu', 'co

1

In [ ]:
from ogb.nodeproppred import Evaluator
from ogb.graphproppred import Evaluator
evaluator = Evaluator(name = 'ogbg-molhiv')
evaluator


In [ ]:
node_features = GraphPropPredDataset(name='ogbg-molhiv')
test_data = b.make_tf_datasets(node_features)['test']


In [ ]:
labels = g.getLabels(test_data)
labels 

len(labels)


4113

In [ ]:
# problem: Ich habe eine letzte Frage: Ich muss um alle metriken zu tracken, die ich in config aufzähle für die jeweiligen datasets meine funktion create_model umschreiben und 'dataset_name' als parameter zusätzlich übergeben und dann config(dataset_name) aufrufen. Und create_grid_models ruft dann nochmal beide methoden auf. Es kommt mir wie richtig schlechter stil vor (normalerweise ist das ja unterwünscht, alles soll so modular wie möglich sein und die 'komplexeste' methode ruft dann alle helper auf, aber ich sehe hier keinen besseren weg, da metriken nur an model.compile übergeben werden können um dann so schön alle auf einmal getrackt zu werden. Was würdest du machen ?


# lösung: ja, den dataset_name weißt du ja wenn du die grid_search startest. die grid_search funktion muss also nur einmal die config berechnen und kann diese config dann an create_model übergeben. mit "das evaluierte dataset" meinte ich jetzt nicht speziell den aufruf einer bestimmten evaluate methode, sondern den generellen experimentellen auswertungsprozess für ein dataset (all inclusive, grid search etc.)




In [ ]:
g.config('ogbg-molhiv')


{'include_mask': False,
 'num_tasks': 1,
 'loss': <keras.losses.BinaryCrossentropy at 0x7f9ab4755e70>,
 'metrics': [keras.metrics.metrics.AUC,
  keras.metrics.metrics.BinaryAccuracy,
  keras.metrics.metrics.Precision,
  keras.metrics.metrics.Recall],
 'evaluator': <ogb.graphproppred.evaluate.Evaluator at 0x7f9ab4754b20>}